In [ ]:
# Colab Python 3.12 + GPU A100
%%capture
!pip uninstall -y torch torchvision torchaudio unsloth unsloth_zoo trl peft accelerate bitsandbytes xformers datasets pandas kagglehub

# Cài torch mới + triton + transformers từ GitHub
!pip install -q --no-cache-dir \
    "torch>=2.8.0" "triton>=3.4.0" \
    git+https://github.com/huggingface/transformers \
    torchvision bitsandbytes

# Cài Unsloth dev version
!pip install -q --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install -q --no-cache-dir unsloth_zoo

# Các package hỗ trợ inference / dataset
!pip install -q trl datasets accelerate peft xformers pandas kagglehub

!pip install pyngrok


In [ ]:
from google.colab import files
import json

uploaded = files.upload()

filename = list(uploaded.keys())[0]

data = []
with open(filename, "r", encoding="utf-8") as f:
    for line in f:
        data.append(json.loads(line))

print(f"Tổng số sample: {len(data)}")
print("Sample đầu tiên:", data[0])


Saving syllabus_dataset.jsonl to syllabus_dataset.jsonl
Tổng số sample: 1563
Sample đầu tiên: {'instruction': 'Xây dựng giáo trình môn học', 'input': 'Mục tiêu khóa học: Nắm vững các nguyên tắc, công cụ và kỹ thuật đảm bảo chất lượng phần mềm  trong phát triển phần mềm .', 'output': {'syllabus': ['Bài 1: Cơ Bản Về Đảm Bảo Chất Lượng', 'Bài 2: Bốn Chiều Hướng Của Chất Lượng', 'Bài 3: Chất Lượng Sản Phẩm phần mềm', 'Bài 4: Môi Trường Tổ Chức Khuyến Khích Văn Hóa Chất Lượng', 'Bài 5: Xác Minh phần mềm', 'Bài 6: Xác Thực', 'Bài 7: Chất Lượng Sản Phẩm phần mềm : Độ Tin Cậy', 'Bài 8: Kiểm Soát Chất Lượng phần mềm', 'Bài 9: Công Cụ Và Kỹ Thuật Đảm Bảo Chất Lượng', 'Bài 10: Thực Hành Đảm Bảo Chất Lượng Trong Dự Án', 'Bài 11: Đo Lường Và Cải Thiện Chất Lượng', 'Phụ lục: Tài Nguyên Và Tham Khảo']}}


In [ ]:
from datasets import Dataset

dataset = Dataset.from_list(data)

dataset_split = dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = dataset_split["train"]
val_dataset = dataset_split["test"]
train_dataset.save_to_disk("/content/giao_an_train")
val_dataset.save_to_disk("/content/giao_an_val")

print(f"Train samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")


Saving the dataset (0/1 shards):   0%|          | 0/1406 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/157 [00:00<?, ? examples/s]

Train samples: 1406
Validation samples: 157


In [ ]:
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments
from datasets import load_from_disk

# Load model
model, tokenizer = FastLanguageModel.from_pretrained(
    "unsloth/Qwen2.5-1.5B-Instruct",
    max_seq_length=2048,
    dtype="bfloat16",
    load_in_4bit=True,
)

# Áp dụng LoRA
model = FastLanguageModel.get_peft_model(model, r=32)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.12.6: Fast Qwen2 patching. Transformers: 4.57.3.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.53G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Unsloth 2025.12.6 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:
def combine_fields(example):
    syllabus_text = "\n".join(example['output']['syllabus'])
    return {"text": f"{example['instruction']}\n{example['input']}\n{syllabus_text}"}

train_ds = train_dataset.map(combine_fields)
val_ds   = val_dataset.map(combine_fields)
train_ds.save_to_disk("/content/giao_an_train")
val_ds.save_to_disk("/content/giao_an_val")


Map:   0%|          | 0/1406 [00:00<?, ? examples/s]

Map:   0%|          | 0/157 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1406 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/157 [00:00<?, ? examples/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:

LORA_DIR = "/content/drive/MyDrive/edtech-syllabus-lora"

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    dataset_text_field="text",
    max_seq_length=2048,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=8,
        num_train_epochs=3,
        learning_rate=5e-5,
        bf16=True,
        logging_steps=10,
        output_dir=LORA_DIR,
        optim="paged_adamw_8bit",
        report_to="none",
        save_strategy="steps",
        save_steps=500,
    ),
    packing=True,
)


Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/1406 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/157 [00:00<?, ? examples/s]

In [ ]:

# Thư mục lưu model trên Drive
LORA_DIR = "/content/drive/MyDrive/edtech-syllabus-lora"

print("ĐANG TRAIN LoRA – CHỈ 30 PHÚT!")
trainer.train()

# Lưu model LoRA trực tiếp vào Drive
trainer.save_model(LORA_DIR)
tokenizer.save_pretrained(LORA_DIR)

print(f"Đã lưu model LoRA vào {LORA_DIR}")


The model is already on multiple devices. Skipping the move to device specified in `args`.


ĐANG TRAIN LoRA – CHỈ 30 PHÚT!


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,406 | Num Epochs = 3 | Total steps = 264
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 8 x 1) = 16
 "-____-"     Trainable parameters = 36,929,536 of 1,580,643,840 (2.34% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,1.904700
20,1.805700
30,1.694700
40,1.622900
50,1.536800
60,1.601000
70,1.493200
80,1.473500
90,1.429700
100,1.394000


Đã lưu model LoRA vào /content/drive/MyDrive/edtech-syllabus-lora


In [ ]:
from unsloth import FastLanguageModel

# Load LoRA đã train
model, tokenizer = FastLanguageModel.from_pretrained(
    LORA_DIR,
    max_seq_length=2048,
    dtype="bfloat16",
    load_in_4bit=True,
)

def generate_syllabus(instruction):
    prompt = f"{instruction}\n"
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.cuda()
    output_ids = model.generate(input_ids, max_length=1024)
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Test
instruction = "Tạo giáo án lập trình Python cho người mới bắt đầu, chia theo tuần, có mục tiêu và nội dung chi tiết."
syllabus = generate_syllabus(instruction)
print(syllabus)


==((====))==  Unsloth 2025.12.5: Fast Qwen2 patching. Transformers: 4.57.3.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Tạo giáo án lập trình Python cho người mới bắt đầu, chia theo tuần, có mục tiêu và nội dung chi tiết.
Mục tiêu: Học cơ bản lập trình hướng đối tượng với Python 3.10.

Thứ 1: Giới thiệu về Python (2 giờ)
Bài 1: Cài đặt PyCharm và quản lý project (45 phút)
Bài 2: Các loại biến và tính toán (45 phút)

Thứ 2: Kiến trúc lập trình (4 giờ)
Bài 3: Class vs Function trong OOP (60 phút)
Bài 4: Private, Public, Protected (60 phút)

Thứ 3: Inheritance & Composition (4 giờ)
Bài 5: Super() và Property Decorator (60 phút)
Bài 6: Polymorphism (Overloading) (60 phút)

Thứ 4: Interface và Abstract Classes (4 giờ)
Bài 7: ABC Modules (60 phút)
Bài 8: Abstract Factory Pattern (60 phút)

Thứ 5: Testing với unittest (4 giờ)
Bài 9: Mocking và Stubs (60 phút)
Bài 10: Unit Testing (60 phút)

Thứ 6: Dự Án: Quản Lý Bảng Điểm Học Tập (4 giờ)
Bài 11: Dữ liệu từ File (60 phút)
Bài 12: Final Project: Học tập tự động hóa điểm (60 phút)


In [ ]:
instruction = "Tạo giáo án Machine learning."
syllabus = generate_syllabus(instruction)
print(syllabus)


Tạo giáo án Machine learning.
Mục tiêu khóa học: Thành thạo các thuật toán và công cụ machine learning từ cơ bản đến nâng cao, bao gồm supervised learning (regression, classification), unsupervised learning (clustering) và reinforcement learning.
Chương 1: Cơ Bản Mạng Máy Tính (30 giờ)
Bài 1: Nâng Cao Python với NumPy và Pandas (5 giờ)
Bài 2: Các Library ML: Scikit-Learn, TensorFlow, PyTorch (8 giờ)
Bài 3: Dự Án Xây Dự Trong Mô hình (8 giờ)
Chương 2: Học Lập Trình (40 giờ)
Bài 4: Thực Hành Supervised Learning: Linear Regression, Logistic Regression (8 giờ)
Bài 5: Non-Linear Model: Decision Tree Classifier, SVM (9 giờ)
Bài 6: Unsupervised Learning: Clustering (K-Means, Hierarchical) (9 giờ)
Chương 3: Nâng Cao (50 giờ)
Bài 7: Reinforcement Learning: DQN, PPO (10 giờ)
Bài 8: Dự Án: Dự Án Sales Forecasting, Image Classification (25 giờ)
Bài 9: Dự Án: Build Recommendation System (Collaborative Filtering, Content-Based Filtering) (10 giờ)
Bài 10: Dự Án: Predict Stock Price, Fraud Detection (

In [ ]:
from unsloth import FastLanguageModel
from peft import PeftModel
import torch
from google.colab import drive
drive.mount('/content/drive')

LORA_DIR = "/content/drive/MyDrive/edtech-syllabus-lora"

# Load model gốc
model, tokenizer = FastLanguageModel.from_pretrained(
    "unsloth/Qwen2.5-1.5B-Instruct",
    max_seq_length=2048,
    dtype="bfloat16",
    load_in_4bit=True
)

# Load LoRA đã train
model = PeftModel.from_pretrained(model, LORA_DIR)
model.eval()
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

def generate_syllabus(instruction: str, max_new_tokens=1024):
    # Tokenize instruction
    inputs = tokenizer(instruction, return_tensors="pt").to(device)

    # Sinh text
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=True)

    # Decode ra string
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test
instruction = "Tạo giáo án lập trình NestJS."
syllabus = generate_syllabus(instruction)
print(syllabus)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Mounted at /content/drive
==((====))==  Unsloth 2025.12.6: Fast Qwen2 patching. Transformers: 4.57.3.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.53G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Tạo giáo án lập trình NestJS. Mục tiêu: Lập trình hệ thống RESTful với TypeScript và Express.
Chương 1: Kiến trúc NestJS
Bài 1: Dependency Injection cơ bản
Bài 2: Module và Controller
Chương 2: Model và Service
Bài 3: Entity (Mongoose/TypeScript)
Bài 4: Service thực thi logic
Chương 3: Websockets và Event Emitter
Bài 5: Kết nối WebSocket
Bài 6: Quản lý sự kiện trong ứng dụng
Chương 4: Security
Bài 7: JWT Authentication
Bài 8: Rate Limiting
Chương 5: Testing
Bài 9: Unit testing với Koa Test
Bài 10: Integration test
Chương 6: Database
Bài 11: Mongodb + TypeORM
Bài 12: Migrations cơ bản
Chương 7: API Gateway
Bài 13: Tránh trùng lặp code với Nest Gateway
Chương 8: Microservices
Bài 14: Chia nhỏ ứng dụng thành các microservice
Chương 9: Deployment
Bài 15: Dockerfile chuẩn NestJS
Bài 16: CI/CD với GitHub Actions hoặc CircleCI
Phụ lục: Xử lý Error Handling chuyên sâu
Chương 10: Dự án cuối: Hệ thống quản lý khóa học Online
Bài 17: Quản lý tài khoản học viên
Bài 18: Giao diện người dùng đơn giả

In [ ]:
# Test
instruction = "Tạo giáo án lập trình python."
syllabus = generate_syllabus(instruction)
print(syllabus)

Tạo giáo án lập trình python. Mục tiêu khóa học: Học lập trình hướng đối tượng với Python.
Chương 1: Giới thiệu về OOP (ước lượng 20 giờ)
Bài 1: Khái niệm lớp và đối tượng (4 giờ)
Bài 2: Getter, Setter, Destructor (5 giờ)
Bài 3: Polymorphism và Inheritance (5 giờ)
Bài 4: Abstract Classes và Interfaces (6 giờ)
Chương 2: Kiến trúc Dữ liệu (ước lượng 25 giờ)
Bài 5: Class Hierarchy và Encapsulation (6 giờ)
Bài 6: Polymorphic Methods và Operator Overloading (7 giờ)
Chương 3: Nâng Cao (ước lượng 25 giờ)
Bài 7: Decorators và Context Managers (8 giờ)
Bài 8: Multithreading với Python (7 giờ)
Phụ lục: Dự Án Quản Lý Tài Khoản (5 giờ)


In [ ]:
# 1️⃣ Cài Flask
!pip install flask flask-ngrok --quiet


In [ ]:
!pip install -q flask flask-ngrok unsloth transformers torch

In [ ]:
!pip install pyngrok


In [ ]:
# ----------------------------
# Mount Google Drive
# ----------------------------
from google.colab import drive
drive.mount('/content/drive')

# ----------------------------
# Import thư viện
# ----------------------------
from flask import Flask, request, jsonify
from unsloth import FastLanguageModel
from peft import PeftModel
import torch
from threading import Thread

# ----------------------------
# Cấu hình model LoRA
# ----------------------------
LORA_DIR = "/content/drive/MyDrive/edtech-syllabus-lora"

device = "cuda" if torch.cuda.is_available() else "cpu"

# Load model gốc
model, tokenizer = FastLanguageModel.from_pretrained(
    "unsloth/Qwen2.5-1.5B-Instruct",
    max_seq_length=2048,
    dtype="bfloat16",
    load_in_4bit=True,
)

# Load LoRA adapter từ Drive
model = PeftModel.from_pretrained(model, LORA_DIR)
model.to(device)
model.eval()

# ----------------------------
# Hàm sinh giáo án
# ----------------------------
def generate_syllabus(instruction: str, max_len=1024):
    prompt = f"{instruction}"
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    output = model.generate(**inputs, max_new_tokens=max_len)
    text = tokenizer.decode(output[0], skip_special_tokens=True)
    return text

# ----------------------------
# Flask app
# ----------------------------
app = Flask(__name__)

@app.route("/generate", methods=["POST"])
def generate():
    data = request.json
    if "instruction" not in data:
        return jsonify({"error": "Missing 'instruction'"}), 400
    instruction = data["instruction"]
    syllabus_text = generate_syllabus(instruction)
    return jsonify({"syllabus": syllabus_text})

# ----------------------------
# Chạy server trên background thread
# ----------------------------
def run_flask():
    # Chạy trên localhost:5000
    app.run(host="0.0.0.0", port=5000, debug=False)

thread = Thread(target=run_flask)
thread.start()
print("API Flask đang chạy tại http://127.0.0.1:5000")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.12.5: Fast Qwen2 patching. Transformers: 4.57.3.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.53G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

API Flask đang chạy tại http://127.0.0.1:5000


In [ ]:
from pyngrok import ngrok

# Thay bằng token
!ngrok authtoken 36xTR8SEGHgyAzdIZyFi63dtOxc_HAKgA4GBcoudX1NWL5uG

# Mở tunnel từ cổng 5000
public_url = ngrok.connect(5000)
print("Public URL:", public_url)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Public URL: NgrokTunnel: "https://christoper-chronogrammatic-le.ngrok-free.dev" -> "http://localhost:5000"
